In [1]:
from keras.models import Sequential, load_model
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
device=tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(device[0], True)

In [3]:
device

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [168]:
shakespeare_url="https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath=tf.keras.utils.get_file('shakespeare_txt',shakespeare_url)
with open(filepath) as f:
    shakespeare_txt=f.read()

In [171]:
print(shakespeare_txt[0:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [176]:
"".join(sorted(set(shakespeare_txt.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

#Convert the text into Tokens to derive a number from each char

In [177]:
tokenizer_char = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer_char.fit_on_texts([shakespeare_txt]) 
    
print(tokenizer_char.texts_to_sequences(['First']))   #On one example we use texts_to_sequences function 

print(tokenizer_char.sequences_to_texts([[20, 6, 9, 8, 3]]))  #On one example we use sequences_to_texts function

#Number of distinct characters
max_id = len(tokenizer_char.word_index)
print(max_id)

#number of characters
dataset_size=tokenizer_char.document_count
print(dataset_size)

#Encoding each characters by its ID (0 to 38)
[encoded] = np.array(tokenizer_char.texts_to_sequences([shakespeare_txt]))-1
encoded[0:10]

[[20, 6, 9, 8, 3]]
['f i r s t']
39
1


array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5])

In [178]:
dataset_size = encoded.shape[0]
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

#Determine the window size for each segment - First window ranges from 0 to 100, second window ranges from 1 to 101 and so on
n_steps = 100
window_length = n_steps + 1               #Target = Input shifted 1 character
dataset = dataset.window(window_length, shift=1, drop_remainder= True)

#Window method creates a nested dataset and so we need to use flatten map to transform it
dataset=dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

#Gradient Descent works best when data is independent and identically distributed, so data is shuffled
batch_size = 32
dataset=dataset.shuffle(10000).batch(batch_size)

#Divide the dataset into train and target batch window data
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

#Convert the training dataset into one-hot-encoding vector
dataset=dataset.map(lambda X_batch,y_batch: (tf.one_hot(X_batch, depth = max_id), y_batch))
dataset = dataset.prefetch(1)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>

In [179]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


After performing all the preprocessing tasks, now we can train and compile the RNN model and start predicting the output

In [181]:
with tf.device('/GPU:0'):
    model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
    ]) 
    
    model.summary()
    
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam')
    history = model.fit(dataset, epochs=10)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, None, 128)         64896     
                                                                 
 gru_7 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed_3 (TimeDis  (None, None, 39)         5031      
 tributed)                                                       
                                                                 
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
31368/31368 [==============================] - 259s 8ms/step - loss: 1.6200
Epoch 2/10
31368/31368 [==============================] - 265s 8ms/step - loss: 1.5403
Epoch 3/10
31368/31368 [==============================] - 269s 9ms/step 

In [219]:
def preprocess(texts):
    X=np.array(tokenizer_char.texts_to_sequences(texts))-1
    #print(X.shape)
    return tf.one_hot(X,max_id)

X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model.predict(X_new), axis=-1)
print(tokenizer_char.sequences_to_texts(Y_pred+1)[0][-1])

def next_char(text, temperature=1):
    X_new=preprocess([text])
    y_proba=model(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits, num_samples=1)+1
    return tokenizer_char.sequences_to_texts(char_id.numpy())[0]

tf.random.set_seed(42)
#print(next_char("How are yo", temperature=1))

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text,temperature)
    return text

tf.random.set_seed(42)
print(complete_text('t',temperature=0.2))
print('\nTemp 0.5')
print(complete_text('w',temperature=0.5))
print('\nTemp 1')
print(complete_text('w',temperature=1))
print('\nTemp 2')
print(complete_text('w',temperature=2))

u
t the worst.

gremio:
i pray, sir. biondello, sir, 

Temp 0.5
will i she shall be obed with rich and besiders:
th

Temp 1
wpeniom,
that we came all the virtue hid read-fault

Temp 2
which andemen's, tres
fo  asdour to dly beedg;
i' a


In [14]:
tokenizer_word = keras.preprocessing.text.Tokenizer()
tokenizer_word.fit_on_texts([shakespeare_txt]) 

print(tokenizer_word.texts_to_sequences(['First']))

print(tokenizer_word.sequences_to_texts([[88,269]]))

#Number of distinct characters
max_id = len(tokenizer_word.word_index)
print(max_id)

#number of characters
dataset_size=tokenizer_word.document_count
print(dataset_size)

#Encoding each word by its ID
[encoded] = np.array(tokenizer_word.texts_to_sequences([shakespeare_txt])) - 1
[encoded]

[[88]]
['first citizen']
12632
1


[array([  87,  268,  138, ...,   24,  130, 1827])]

In [13]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [14]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
